<CENTER><img src="./ATLASOD.gif" style="width:50%"></CENTER>

# Introduction to Machine Learning using ATLAS Open Data

This notebook uses a cut-down version of ATLAS Open Data http://opendata.atlas.cern to show you the steps to apply Machine Learning in search for the Higgs boson!

ATLAS Open Data provides open access to proton-proton collision data at the LHC for educational purposes. ATLAS Open Data resources are ideal for high-school, undergraduate and postgraduate students.

Notebooks are web applications that allow you to create and share documents that can contain for example:
1. live code
2. visualisations
3. narrative text

This notebook builds on [HZZAnalysis.ipynb](https://github.com/atlas-outreach-data-tools/notebooks-collection-opendata/blob/master/13-TeV-examples/uproot_python/HZZAnalysis.ipynb) and is adapted from [HZZ_NeuralNet_demo.ipynb](https://github.com/atlas-outreach-data-tools/notebooks-collection-opendata/blob/master/13-TeV-examples/uproot_python/HZZ_NeuralNet_demo.ipynb).

HZZAnalysis.ipynb loosely follows the [discovery of the Higgs boson by ATLAS](https://www.sciencedirect.com/science/article/pii/S037026931200857X) (mostly Section 4 and 4.1)

Notebooks are a perfect platform to develop Machine Learning for your work, since you'll need exactly those 3 things: code, visualisations and narrative text!

We're interested in Machine Learning because we can design an algorithm to figure out for itself how to do various analyses, potentially saving us countless human-hours of design and analysis work.

Machine Learning use within ATLAS includes: 
* particle tracking
* particle identification
* signal/background classification
* and more!

This notebook will focus on signal/background classification.

Contents:

1. [Running a Jupyter notebook](#Running-a-Jupyter-notebook) <br />
2. [First time setup on your computer](#First-time-setup-on-your-computer) <br />
3. [To setup everytime](#To-setup-everytime) <br />
4. [Samples](#Samples) <br />
5. [Reading file with uproot](#Reading-file-with-uproot) <br />
6. [Set up the data for the Machine Learning Algorithms](#Set-up-the-data-for-the-Machine-Learning-Algorithms) <br />
   6.1 [The Training and Testing split](#The-Training-and-Testing-split) <br />
7. [Decision Trees](#Decision-Trees) <br />
   7.1 [Receiver Operarting Characteristic (ROC) curve](#Receiver-Operarting-Characteristic-(ROC)-curve) <br />
7. [Going-further](#Going-further) <br />

## Running a Jupyter notebook

To run the whole Jupyter notebook, in the top menu click Cell -> Run All.

To propagate a change you've made to a piece of code, click Cell -> Run All Below.

You can also run a single code cell, by clicking Cell -> Run Cells, or using the keyboard shortcut Shift+Enter.

## First time setup on your computer
This first cell only needs to be run the first time you open this notebook on your computer. 

If you close Jupyter and re-open on the same computer, you won't need to run this first cell again.

In [ ]:
import sys
!{sys.executable} -m pip install --upgrade --user pip # update the pip package installer
!{sys.executable} -m pip install uproot3 pandas numpy matplotlib sklearn --user # install required packages

## To setup everytime
Cell -> Run All Below

to be done every time you re-open this notebook

We're going to be using a number of tools to help us:
* uproot: lets us read .root files typically used in particle physics into data formats used in Machine Learning
* pandas: lets us store data as dataframes, a format widely used in Machine Learning
* numpy: provides numerical calculations such as histogramming
* matplotlib: common tool for making plots, figures, images, visualisations

In [ ]:
import uproot3 # for reading .root files
import pandas as pd # to store data as dataframe
import time # to measure time to analyse
import math # for mathematical functions such as square root
import numpy as np # for numerical calculations such as histogramming
import matplotlib.pyplot as plt # for plotting
from matplotlib.ticker import AutoMinorLocator # for minor ticks

import infofile # local file containing info on cross-sections, sums of weights, dataset IDs
#plt.rcParams['figure.figsize'] = [10, 8]

## Samples

In this notebook we only process the signal <span style="color:blue">H->ZZ</span> and the main background <span style="color:red">ZZ</span>, for illustration purposes. You can add data and the Z and ttbar <span style="color:red">backgrounds</span> after if you wish.

In [ ]:
samples = {

    'ZZ' : {
        'list' : ['llll']
    },

    r'$H \rightarrow ZZ \rightarrow \ell\ell\ell\ell$' : { # H -> ZZ -> llll
        'list' : ['ggH125_ZZ4lep'] # gluon-gluon fusion
    }

}

Define function to get data from files. 

The datasets used in this notebook have already been filtered to include at least 4 leptons per event, so that processing is quicker.

In [ ]:
def get_data_from_files():

    data = {} # define empty dictionary to hold dataframes
    tuple_path = "4lep/" #local
    #tuple_path = "https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/4lep/" # web address
    for s in samples: # loop over samples
        print('Processing '+s+' samples') # print which sample
        frames = [] # define empty list to hold data
        for val in samples[s]['list']: # loop over each file
            if s == 'data': prefix = "Data/" # Data prefix
            else: # MC prefix
                prefix = "/MC/mc_"+str(infofile.infos[val]["DSID"])+"."
            fileString = tuple_path+prefix+val+".4lep.root" # file name to open
            temp = read_file(fileString,val) # call the function read_file defined below
            frames.append(temp) # append dataframe returned from read_file to list of dataframes
        data[s] = pd.concat(frames) # dictionary entry is concatenated dataframes
    
    return data # return dictionary of dataframes

We add functions to return the individual lepton transverse momenta, in GeV

In [ ]:
def calc_lep_pt_i(lep_pt,i):
    return lep_pt[i]/1000 # /1000 to go from MeV to GeV

## Reading file with uproot
If you change anything related to the inputs from the file: Cell -> Run All Below

In [ ]:
def read_file(path,sample):
    start = time.time() # start the clock
    print("\tProcessing: "+sample) # print which sample is being processed
    data_all = pd.DataFrame() # define empty pandas DataFrame to hold all data for this sample
    tree = uproot3.open(path)["mini"] # open the tree called mini
    numevents = uproot3.numentries(path, "mini") # number of events
    for data in tree.iterate(['eventNumber',
                              'lep_charge','lep_type','lep_pt',
                              # uncomment these variables if you want to calculate masses 
                              'lep_eta','lep_phi','lep_E', 
                              # add more variables here if you make cuts on them 
                              'totalWeight'
                             ], # variables to calculate Monte Carlo weight
                             outputtype=pd.DataFrame, # choose output type as pandas DataFrame
                             entrystop=numevents): # process up to numevents*fraction

        # return the individual lepton transverse momenta in GeV
        data['lep_pt_1'] = np.vectorize(calc_lep_pt_i)(data.lep_pt,1)
        data['lep_pt_2'] = np.vectorize(calc_lep_pt_i)(data.lep_pt,2)
        
        nIn = len(data.index) # number of events in this batch
        nOut = len(data.index) # number of events passing cuts in this batch
        data_all = pd.concat([data_all,data]) # append dataframe from this batch to the dataframe for the whole sample
        elapsed = time.time() - start # time taken to process
        print("\t\t nIn: "+str(nIn)+",\t nOut: \t"+str(nOut)+"\t in "+str(round(elapsed,1))+"s") # events before and after
    
    return data_all # return dataframe containing events passing all cuts

This is where the processing happens (this will take a few seconds)

In [ ]:
start = time.time() # time at start of whole processing
data = get_data_from_files() # process all files
elapsed = time.time() - start # time after whole processing
print("Time taken: "+str(round(elapsed,1))+"s") # print total time taken to process every file

To see what is in the background (ZZ) dataframe run the cell below:

In [ ]:
print(data['ZZ'])

Here we define histograms for the variables that we'll look to train the MVAs with:

In [ ]:
lep_pt_2 = { # dictionary containing plotting parameters for the lep_pt_2 histogram
    # change plotting parameters
    'bin_width':1, # width of each histogram bin
    'num_bins':50, # number of histogram bins
    'xrange_min':10, # minimum on x-axis
    'xlabel':r'$lep\_pt$[2] [GeV]', # x-axis label
}

lep_pt_1 = { # dictionary containing plotting parameters for the lep_pt_1 histogram
    # change plotting parameters
    'bin_width':1, # width of each histogram bin
    'num_bins':50, # number of histogram bins
    'xrange_min':10, # minimum on x-axis
    'xlabel':r'$lep\_pt$[1] [GeV]', # x-axis label
}

SoverB_hist_dict = {'lep_pt_2':lep_pt_2,'lep_pt_1':lep_pt_1} 
# add a histogram here if you want it plotted

This cell is just to put some nice ATLAS Open Data labels on the plots:

In [ ]:
def opendatatext(ax):
    # Add text 'ATLAS Open Data' on plot
    plt.text(0.05, # x
             0.93, # y
             'ATLAS Open Data', # text
             transform=ax.transAxes, # coordinate system used is that of distributions_axes
             fontsize=13 ) 
    # Add text 'for education' on plot
    plt.text(0.05, # x
             0.88, # y
             'for education', # text
             transform=ax.transAxes, # coordinate system used is that of distributions_axes
             style='italic',
             fontsize=8 )  
    return

Here we define a function to illustrate the optimum cut value on individual variables, based on <span style="color:blue">signal</span> to <span style="color:red">background</span> ratio.

In [ ]:
def plot_SoverB(data):
    
    signal = r'$H \rightarrow ZZ \rightarrow \ell\ell\ell\ell$' # which sample is the signal

    # *******************
    # general definitions (shouldn't need to change)

    for x_variable,hist in SoverB_hist_dict.items(): # access the dictionary of histograms defined in the cell above

        h_bin_width = hist['bin_width'] # get the bin width defined in the cell above
        h_num_bins = hist['num_bins'] # get the number of bins defined in the cell above
        h_xrange_min = hist['xrange_min'] # get the x-range minimum defined in the cell above
        h_xlabel = hist['xlabel'] # get the x-axis label defined in the cell above
    
        bin_edges = [ h_xrange_min + x*h_bin_width for x in range(h_num_bins+1) ] # bin limits
        bin_centres = [ h_xrange_min+h_bin_width/2 + x*h_bin_width for x in range(h_num_bins) ] # bin centres
        
        signal_x = data[signal][x_variable] # histogram the signal
    
        mc_x = [] # define list to hold the Monte Carlo histogram entries

        for s in samples: # loop over samples
            if s not in ['data', signal]: # if not data nor signal
                mc_x = [*mc_x, *data[s][x_variable] ] # append to the list of Monte Carlo histogram entries

    
    
        # *************
        # Signal and background distributions
        # *************
        distributions_axes = plt.gca() # get current axes
 
        mc_heights = distributions_axes.hist(mc_x, bins=bin_edges, color='red', 
                                             label='Total background',
                                             histtype='step', # lineplot that's unfilled
                                             density=True ) # normalize to form probability density
        signal_heights = distributions_axes.hist(signal_x, bins=bin_edges, color='blue',
                                                 label=signal, 
                                                 histtype='step', # lineplot that's unfilled
                                                 density=True, # normalize to form probability density
                                                 linestyle='--' ) # dashed line
        
        distributions_axes.set_xlim( left=bin_edges[0], right=bin_edges[-1] ) # x-limits of the distributions axes
        distributions_axes.set_ylabel('Arbitrary units' ) # y-axis label for distributions axes
        distributions_axes.set_ylim( top=max(signal_heights[0])*1.3 ) # set y-axis limits
        plt.title('Signal and background '+x_variable+' distributions') # add title
        distributions_axes.legend() # draw the legend
        distributions_axes.set_xlabel( h_xlabel ) # x-axis label
        
        opendatatext(ax=plt.gca())
    
        plt.show() # show the Signal and background distributions
    
    
        # *************
        # Signal to background ratio
        # *************
        plt.figure() # start new figure
        SoverB = [] # list to hold S/B values
        for cut_value in bin_edges: # loop over bins
            signal_weights_passing_cut = sum(data[signal][data[signal][x_variable]<cut_value].totalWeight)
            background_weights_passing_cut = 0 # start counter for background weights passing cut
            for s in samples: # loop over samples
                if s not in ['data', signal]: # if not data nor signal
                    background_weights_passing_cut += sum(data[s][data[s][x_variable]<cut_value].totalWeight)
            if background_weights_passing_cut!=0: # some background passes cut
                SoverB_value = signal_weights_passing_cut/background_weights_passing_cut
                SoverB_percent = 100*SoverB_value # multiply by 100 for percentage
                SoverB.append(SoverB_percent) # append to list of S/B values
        
        SoverB_axes = plt.gca() # get current axes
        SoverB_axes.plot( bin_edges[:len(SoverB)], SoverB ) # plot the data points
        SoverB_axes.set_xlim( left=bin_edges[0], right=bin_edges[-1] ) # set the x-limit of the main axes
        SoverB_axes.set_ylabel( 'S/B (%)' ) # write y-axis label for main axes
        plt.title('Signal to background ratio for different '+x_variable+' cut values', family='sans-serif')
        SoverB_axes.set_xlabel( h_xlabel ) # x-axis label 
        
        plt.show() # show S/B plot
    
    return

Here we call our function to illustrate the optimum cut value on individual variables, based on <span style="color:blue">signal</span> to <span style="color:red">background</span> ratio.

We're not doing any Machine Learning yet! We're looking at the variables we'll later use for Machine Learning.

Let's talk through the lep_pt_2 plots.
1. Imagine placing a cut at 20 GeV in the distributions of <span style="color:blue">signal</span> and <span style="color:red">background</span> (1st plot). This means keeping all events less than 11 GeV in the <span style="color:blue">signal</span> and <span style="color:red">background</span> histograms. 
2. We then take the ratio of the number of <span style="color:blue">signal</span> events that pass this cut, to the number of <span style="color:red">background</span> events that pass this cut. This gives us a starting value for S/B (2nd plot). 
3. We then increase this cut value to 12 GeV, 13 GeV, 14 GeV, etc. Cuts at these values are throwing away more <span style="color:red">background</span> than <span style="color:blue">signal</span>, so S/B increases. 
4. There comes a point around 26 GeV where we start keeping too much <span style="color:blue">background</span>, thus S/B starts to decrease. 
5. Our goal is to find the maximum in S/B, and place the cut there.

The same logic applies to lep_pt_1.

In [ ]:
plot_SoverB(data)

In the [ATLAS Higgs discovery paper](https://www.sciencedirect.com/science/article/pii/S037026931200857X), there are a number of numerical cuts applied, not just on lep_pt_1 and lep_pt_2.

Imagine having to separately optimise about 7 variables! Not to mention that applying a cut on one variable could change the distribution of another, which would mean you'd have to re-optimise... Nightmare.

This is where a Machine Learning algorithms can come to the rescue; they can optimise all variables at the same time.

A ML algorithms not only optimises cuts, but can find correlations in many dimensions that will give better signal/background classification than individual cuts ever could.

That's the end of the introduction to why one might want to use machine learning. If you'd like to try using one, just keep reading below!

## Set up the data for the Machine Learning Algorithms

Choose variables for use in the algorithms

In [ ]:
data_for_NN = {} # define empty dictionary to hold dataframes that will be used to train the NN
NN_inputs = ['lep_pt_1','lep_pt_2'] # list of features for Neural Network
for key in data: # loop over the different keys in the dictionary of dataframes
    data_for_NN[key] = data[key][NN_inputs].copy()
data_for_NN

 Organise data ready for the NN

In [ ]:
# for sklearn data is usually organised                                                                                                                                           
# into one 2D array of shape (n_samples x n_features)                                                                                                                             
# containing all the data and one array of categories                                                                                                                             
# of length n_samples  

all_MC = [] # define empty list that will contain all features for the MC
for key in data: # loop over the different keys in the dictionary of dataframes
    if key!='data': # only MC should pass this
        all_MC.append(data_for_NN[key]) # append the MC dataframe to the list containing all MC features
#X = np.concatenate(all_MC) # concatenate the list of MC dataframes into a single 2D array of features, called X
X = pd.concat(all_MC)

all_y = [] # define empty list that will contain labels whether an event in signal or background
for key in data: # loop over the different keys in the dictionary of dataframes
    if key!=r'$H \rightarrow ZZ \rightarrow \ell\ell\ell\ell$' and key!='data': # only background MC should pass this
        all_y.append(np.zeros(data_for_NN[key].shape[0])) # background events are labelled with 0
all_y.append(np.ones(data_for_NN[r'$H \rightarrow ZZ \rightarrow \ell\ell\ell\ell$'].shape[0])) # signal events are labelled with 1
y = np.concatenate(all_y) # concatenate the list of lables into a single 1D array of labels, called y

### The Training and Testing split
One of the first things to do is split your data into a training and testing set. This will split your data into train-test sets: 90%-10%. It will also shuffle entries so you will not get the first 90% of <span style="color:orange">X</span> for training and the last 33% for testing. This is particularly important in cases where you load all <span style="color:blue">signal</span> events first and then the <span style="color:red">background</span> events.

Here we split our data into two independent samples. The split is to create a training and testing set. The first will be used for training the classifier and the second to evaluate its performance.

We don't want to test on events that we used to train on, this prevents overfitting to some subset of data so the network would be good for the test data but much worse at any *new* data it sees.

In [ ]:
from sklearn.model_selection import train_test_split

# make train and test sets
X_train,X_test, y_train,y_test = train_test_split(X, y, 
                                                  test_size=0.1, 
                                                  random_state=492 )

## Decision Trees
We'll use SciKit Learn (sklearn) in this tutorial. Other possible tools include keras and pytorch. 
There will be a few different version to try out; basic Decision Tree classifiers as well as AdaBoost and Gradient Booster Decesion Tree classifiers.

Several hyper-parameters are set to non default values, but it is suggested that you play around with some of the options and see how it affects the results.

First let's import some of the classifiers we need from sklearn:

In [ ]:
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

The training of a basic Decision Tree classifier with a maximum depth of 3 is done in the next cell.

In [ ]:
dt = tree.DecisionTreeClassifier(max_depth=3)
dt.fit(X_train.values,y_train)

We are then able to plot the "decisions" made at each branching point and see what type of node they result in:

In [ ]:
tree.plot_tree(dt,feature_names=['x','y'],class_names=['sig','bkg'],filled=True)

### Receiver Operarting Characteristic (ROC) curve
A useful plot to judge the performance of a classifier is to look at the ROC curve directly.

This compares the true positive rate (signal classified as signal) to the false positve rate (background classified as signal). The better the performance of the classifier, the closer to the top left the curve will be. A random guess for two would produce the diagonal dashed line on the plot.

In [ ]:
def calc_roc(clf,X,y):
    from sklearn.metrics import roc_curve, auc
    y_score = clf.predict_proba(X.values)[:,1]
    
    fpr, tpr, _  = roc_curve(y_test,y_score)
    roc_auc = auc(fpr,tpr)
    lw = 2
    plt.plot(
        fpr,
        tpr,
        color="darkorange",
        lw=lw,
        label="ROC curve (area = %0.2f)" % roc_auc,
    )
    plt.plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("Receiver operating characteristic")
    plt.legend(loc="lower right")
    opendatatext(plt.gca())
    plt.show()
    return

We can plot the ROC curve for the trained Decision Tree classifier with the next cell:

In [ ]:
calc_roc(dt,X_test,y_test)

In [ ]:
def evaluate_mva(clf,X,y,var1,var2,xmin,xmax):
    sig = pd.DataFrame()
    bkg = pd.DataFrame()
    sig[NN_inputs] = [i for i,j in zip(X.values,y) if j==1.0]
    bkg[NN_inputs] = [i for i,j in zip(X.values,y) if j==0.0]
    sigsamp = sig.sample(n=200)
    bkgsamp = bkg.sample(n=200)
    
    plt.plot(bkgsamp[var1],bkgsamp[var2], 'o', c='tab:orange', label='bkg', alpha=0.5, markeredgecolor='k')
    plt.plot(sigsamp[var1],sigsamp[var2], 'o', c='tab:blue', label='sig', alpha=0.5, markeredgecolor='k')
    plt.xlim(xmin,xmax)
    plt.ylim(xmin,xmax)
    ax = plt.gca()
    from sklearn.inspection import DecisionBoundaryDisplay
    DecisionBoundaryDisplay.from_estimator(clf, X.values, ax=ax, cmap=plt.cm.PuOr, alpha=0.8, eps=0.5, response_method="predict_proba")
    opendatatext(ax)
    plt.legend(loc='upper right')
    plt.show()
    return

In [ ]:
evaluate_mva(dt,X_test,y_test,'lep_pt_1','lep_pt_2',10,60)

In [ ]:
ABDT = AdaBoostClassifier(tree.DecisionTreeClassifier(max_depth=4),
                         algorithm="SAMME",
                         n_estimators=20)
ABDT.fit(X_train.values,y_train)

In [ ]:
evaluate_mva(ABDT,X_test,y_test,'lep_pt_1','lep_pt_2',10,60)

In [ ]:
def errorVsTree(clf,X_train,y_train,X_test,y_test):
    train_errors = []
    test_errors  = []
    
    for train_predict,test_predict in zip(clf.staged_predict(X_train.values),clf.staged_predict(X_test.values)):
        train_errors.append(1. - accuracy_score(train_predict, y_train))
        test_errors.append(1. - accuracy_score(test_predict, y_test))

    n_trees = len(clf)

    plt.plot(range(1, n_trees + 1), train_errors, c='red', label='Train')
    plt.plot(range(1, n_trees + 1), test_errors, c='blue', label='Test')
    plt.legend(loc='upper right')
    plt.ylim(0.9*min(min(train_errors),min(test_errors)),1.1*max(max(train_errors),max(test_errors)))
    plt.ylabel('Test Error')
    plt.xlabel('Number of Trees')
    opendatatext(plt.gca())
    plt.show()

    return

In [ ]:
errorVsTree(ABDT,X_train,y_train,X_test,y_test)

In [ ]:
calc_roc(ABDT,X_test,y_test)

In [ ]:
GBDT = GradientBoostingClassifier(max_depth=4,learning_rate=0.1,
                               n_estimators=20,random_state=0)
GBDT.fit(X_train.values,y_train)

In [ ]:
errorVsTree(GBDT,X_train,y_train,X_test,y_test)

In [ ]:
evaluate_mva(GBDT,X_test,y_test,'lep_pt_1','lep_pt_2',10,60)

In [ ]:
calc_roc(GBDT,X_test,y_test)

### Data Preprocessing for Neural Network

The neural network in Python may have difficulty converging before the maximum number of iterations allowed if the data is not standardised. Multi-layer Perceptron is sensitive to feature scaling, so it is highly recommended to scale your data. Note that you must apply the same scaling to the test set for meaningful results. There are a lot of different methods for standardization of data, we will use the built-in StandardScaler this time.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() # initialise StandardScaler

# Fit only to the training data
scaler.fit(X_train)
# Now apply the transformations to the data:
scaled_X_train = pd.DataFrame()
scaled_X_test = pd.DataFrame()
scaled_X = pd.DataFrame()
scaled_X_train[NN_inputs] = scaler.transform(X_train[NN_inputs])
scaled_X_test[NN_inputs] = scaler.transform(X_test[NN_inputs])
scaled_X[NN_inputs] = scaler.transform(X[NN_inputs])

In [ ]:
from sklearn.neural_network import MLPClassifier
hidden_layer_sizes = [3] # 1 hidden layer
mlp = MLPClassifier(hidden_layer_sizes=(hidden_layer_sizes), # define parameters for our multi-layer-perceptron
                    max_iter=200 )# max number of iterations

In [ ]:
mlp.fit(scaled_X_train.values,y_train)

In [ ]:
evaluate_mva(mlp,scaled_X_test,y_test,'lep_pt_1','lep_pt_2',-2.0,2.0)

In [ ]:
calc_roc(mlp,scaled_X_test,y_test)

## BDTs and NNs - Going further

If you want to go further, there are a number of things you could try:
* Modify some hyper-parameters for the various trainings.
* Add some more variables into the classifiers. Add them in one at a time, rather than all at once, because adding a variable could decrease performance, due to anti-correlation. For some ideas of variables, you can look at the paper for the [discovery of the Higgs boson by ATLAS](https://www.sciencedirect.com/science/article/pii/S037026931200857X) (mostly Section 4 and 4.1).

With each change, keep an eye on the:
* total area under the ROC curve, 
* separation between <span style="color:blue">signal</span> and <span style="color:red">background</span> in the Neural Network output distribution
* S/B scores that can be achieved

## General Techniques

In [ ]:
def plot_output(clf,X,y):
    n_bins = 40
    
    sig = pd.DataFrame()
    bkg = pd.DataFrame()
    
    sig[NN_inputs] = [i for i,j in zip(X.values,y) if j==1.0]
    bkg[NN_inputs] = [i for i,j in zip(X.values,y) if j==0.0]
    
    sig_output = clf.predict_proba(sig.values)[:,1]
    bkg_output = clf.predict_proba(bkg.values)[:,1]
    
    d_min = min(sig_output.min(),bkg_output.min())
    d_max = max(sig_output.max(),bkg_output.max())
    
    plt.hist(bkg_output,bins=n_bins,range=(d_min,d_max), color='tab:orange', label='bkg train',alpha=0.6, density=True)
    plt.hist(sig_output,bins=n_bins,range=(d_min,d_max), color='tab:blue', label='sig train', alpha=0.6, density=True)
    opendatatext(plt.gca())
    plt.ylim([0,plt.gca().get_ylim()[1]*1.25])
    #print(plt.gca().get_ylim())
    plt.legend(loc="upper right")
    plt.show()

    return

In [ ]:
plot_output(mlp,scaled_X_test,y_test)

### Overtraining Checks
Comparing the ML algorithm's output distribution for the training and testing set is a popular way in HEP to check for overtraining. The <span style="color:orange">plot_compare_outputs()</span> method will plot the shape of the algorithm's decision function for each class, as well as overlaying it with the decision function in the training set.

There are techniques to prevent overtraining.
The output also performs the two-sample Kolmogorov-Smirnov test for goodness of fit between the training and testing samples for both the signal and background.
The idea is to compare the underlying distributions of two independent samples, the training and the testing, and if the KS statistic is large, then the p-value will be small, and this suggests that the two do not match well and therefore there may be evidence of overtraining in the classifier.

To try and see and example of this go back to one of the BDTs and increase the depth or the number of estimators, and retrain to see how the output distributions compare for different values.

In [ ]:
def plot_compare_outputs(clf,X_train,y_train,X_test,y_test):
    from scipy.stats import ks_2samp
    n_bins = 30
    
    sig_train = pd.DataFrame()
    bkg_train = pd.DataFrame()
    sig_test = pd.DataFrame()
    bkg_test = pd.DataFrame()
    sig_train[NN_inputs] = [i for i,j in zip(X_train.values,y_train) if j==1.0]
    bkg_train[NN_inputs] = [i for i,j in zip(X_train.values,y_train) if j==0.0]
    sig_test[NN_inputs] = [i for i,j in zip(X_test.values,y_test) if j==1.0]
    bkg_test[NN_inputs] = [i for i,j in zip(X_test.values,y_test) if j==0.0]
    
    sig_train_output = clf.predict_proba(sig_train.values)[:,1]
    bkg_train_output = clf.predict_proba(bkg_train.values)[:,1]

    sig_test_output = clf.predict_proba(sig_test.values)[:,1]
    bkg_test_output = clf.predict_proba(bkg_test.values)[:,1]
    
    d_min = min(sig_train_output.min(),bkg_train_output.min())
    d_max = max(sig_train_output.max(),bkg_train_output.max())
    
    sig_tr,bins,_ = plt.hist(bkg_train_output,bins=n_bins,range=(d_min,d_max), color='tab:orange', label='bkg train',alpha=0.6, density=True)
    bkg_tr,_,_ = plt.hist(sig_train_output,bins=n_bins,range=(d_min,d_max), color='tab:blue', label='sig train', alpha=0.6, density=True)

    bin_centers = (bins[:-1]+bins[1:])/2
    sig_te,_ = np.histogram(sig_test_output,bins=bins,density=True)
    bkg_te,_ = np.histogram(bkg_test_output,bins=bins,density=True)

    plt.plot(bin_centers,bkg_te, 'o', c='tab:orange', label='bkg test', alpha=0.9, markeredgecolor='k')
    plt.plot(bin_centers,sig_te, 'o', c='tab:blue', label='sig test', alpha=0.9, markeredgecolor='k')
    opendatatext(ax=plt.gca())
    plt.ylim([0,plt.gca().get_ylim()[1]*1.4])
    print("Signal:",ks_2samp(sig_tr,sig_te))
    print("Background:",ks_2samp(bkg_tr,bkg_te))
    plt.legend(loc="upper right")
    plt.show()

    return

In [ ]:
plot_compare_outputs(mlp,scaled_X_train,y_train,scaled_X_test,y_test)

In [ ]:
plot_compare_outputs(ABDT,X_train,y_train,X_test,y_test)

In [ ]:
def errorVsSize(clf,cv,X,y,njobs,train_sizes=np.linspace(.1, 1.0, 10)):
    from sklearn.utils import shuffle
    from sklearn.model_selection import learning_curve
    
    X,y = shuffle(X,y)

    train_sizes, train_scores, test_scores = learning_curve(clf, X, y, cv=cv, n_jobs=njobs, train_sizes=train_sizes)
    
    train_errors_mean = np.mean(train_scores, axis=1)
    train_errors_std = np.std(train_scores, axis=1)
    test_errors_mean = np.mean(test_scores, axis=1)
    test_errors_std = np.std(test_scores, axis=1)

    plt.figure()
    plt.xlabel("Training Samples")
    plt.ylabel("1-Error")

    plt.grid()
    plt.fill_between(train_sizes, train_errors_mean - train_errors_std,
                     train_errors_mean + train_errors_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_errors_mean - test_errors_std,
                     test_errors_mean + test_errors_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_errors_mean, 'o-', color="r",
             label="Training Error")
    plt.plot(train_sizes, test_errors_mean, 'o-', color="g",
             label="Test Error")

    plt.legend(loc="best")
    plt.show()
    
    return

In [ ]:
from sklearn.model_selection import ShuffleSplit
ABDT = AdaBoostClassifier(tree.DecisionTreeClassifier(max_depth=6),
                         algorithm="SAMME",
                         n_estimators=20)

cv = ShuffleSplit(n_splits=4, test_size=0.2, random_state=0)
errorVsSize(ABDT,cv,X_train,y_train,4)

## General Techniques - Going further

If you want to go further, there are a number of things you could try:
* Try comparing the different ML algorithms with different numbers of folds for the cross-validation.
* Add in the other <span style="color:blue">H->ZZ signal</span> samples in '[Samples](#samples)'. You can copy them from [HZZAnalysis.ipynb](https://github.com/atlas-outreach-data-tools/notebooks-collection-opendata/blob/master/13-TeV-examples/uproot_python/HZZAnalysis.ipynb) and will need to change the `get_data_from_files()` function to read the web-based files. Try adding them one at a time first, then see how things look with all added.
* Add in real data in '[Samples](#samples)' and see whether the Neural Network output distributions in data and simulation match. You can copy data from [HZZAnalysis.ipynb](https://github.com/atlas-outreach-data-tools/notebooks-collection-opendata/blob/master/13-TeV-examples/uproot_python/HZZAnalysis.ipynb) and again will need to change to read the web-based files. 

Again with each change, keep an eye on the:
* total area under the ROC curve, 
* separation between <span style="color:blue">signal</span> and <span style="color:red">background</span> in the Neural Network output distribution
* S/B scores that can be achieved

Notice that we've trained and tested our ML algortihms on simulated data. We would then *apply* it to real experimental data. Once you're happy with classifiers, you may want to put it back into a full analysis to run over all data.